In [323]:
import pandas as pd
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
import numpy as np

In [324]:
crawling_result = pd.read_csv('인공지능_문화체육관광부_클라썸_crawled.csv')

In [325]:
# crawling_result

In [326]:
query = '인공지능'
business_name = '클라썸'

In [327]:
crawling_result['checker'] = crawling_result.crawling_trg.str.find(business_name)

bussiness_news = crawling_result[crawling_result.checker > -1].copy().reset_index(drop=True)
bussiness_news['idx_original'] = range(len(bussiness_news))

org_news = crawling_result[crawling_result.checker == -1].copy().reset_index(drop=True)
org_news['idx_original'] = range(len(org_news))

business_news_contents = bussiness_news[['idx_original', 'crawling_trg', 'pubDate', 'title', 'content', 'link']]
org_news_contents = org_news[['idx_original', 'crawling_trg', 'pubDate', 'title', 'content', 'link']]

In [328]:
def make_bow(query, articles):
    name = articles.crawling_trg.unique()[0].split(query)[0].strip()

    articles = articles.content.apply(lambda x: x.split()).to_list().copy()
    articles = [[word if not (query in word) else query for word in article] for article in articles]
    articles = [[word if not (name in word) else name for word in article] for article in articles]

    dct = Dictionary(articles)  # fit dictionary
    bow_articles = [dct.doc2bow(article) for article in articles]  # convert corpus to BoW format
    bow_articles = [{k:v for k, v in bow_article} for bow_article in bow_articles]
    
    return name, dct, bow_articles

In [329]:
def id_by_tf_retrieve(query, name, dct, bow_articles):
    query_id = dct.token2id[query]
    query_tfs = []
    for bow_article in bow_articles:
        query_tfs.append(bow_article[query_id])
    query_tfs = np.array(query_tfs)

    name_id = dct.token2id[name]
    name_tfs = []
    for bow_article in bow_articles:
        name_tfs.append(bow_article[name_id])
    name_tfs = np.array(name_tfs)
    
    tfs = query_tfs + name_tfs
    doc_id = np.argmax(tfs)
    # print(tfs)
    return doc_id

In [330]:
def show_article(doc_id, articles):
    print(articles.iloc[doc_id].crawling_trg)
    print(articles.iloc[doc_id].title)
    print(articles.iloc[doc_id].content)

In [331]:
org_news_contents_splits = []
for org in org_news_contents.crawling_trg.unique():
    org_news_contents_split = org_news_contents[org_news_contents.crawling_trg == org].reset_index(drop=True).copy()
    org_news_contents_splits.append(org_news_contents_split)

In [332]:
for articles in org_news_contents_splits:
    name, dct, bow_articles = make_bow(query, articles)
    doc_id = id_by_tf_retrieve(query, name, dct, bow_articles)
    show_article(doc_id, articles)
    print()

국립국어원 인공지능
[출근길 인터뷰] 고마워 한글…2022 한글 주간 개막
[앵커]올해로 576돌을 맞는 한글날을 맞아 '고마워 한글'이라는 주제로 오늘부터 일주일간 다채로운 한글 주간 행사가 열립니다. 국립국어원에서는 '세상을 잇는 한국어, 미래를 이끄는 한글'이라는 주제로 국제학술대회를 개최하는데요. 오늘 출근길 인터뷰에서는 장소원 국립국어원장을 만나봅니다. 박서휘 캐스터 전해주시죠.[캐스터] 출근길인터뷰입니다. 오늘은 장소원 국립국어원장을 만나 얘기 나눠보겠습니다. 안녕하세요.[장소원 국립국어원장]안녕하세요.[캐스터] 쉽고 편리한 한글이 우리에게 주는 고마움을 되새기고자 고마워 한글이라는 주제로 오늘부터 전국 곳곳에서 행사가 열리는데요. 어떤 의미를 담은 행사인지 소개해 주시죠.[장소원 국립국어원장]오늘부터 다음 주 월요일인 10월 10일까지 한글날을 기념하는 2022 한글주관 행사가 국립한글박물관을 비롯한 전국 각지에서 열립니다. 특히 올해는 쉽고 편리한 한글에 대한 고마움을 되새기고자 '고마워 한글'이라는 주제로 국립국어원, 국립한글박물관, 세종학당 재단 등의 기관과 여러 문화 단체들이 참여해서 전야제와 학술대회, 한글 한국어 산업전, 한국문화 체험, 연수 같은 행사를 준비하고 있습니다.[캐스터] 특히 국립국어원에서는 한국어 관련 학자들이 모이는 학술대회뿐 아니라 다양한 행사들을 준비하고 있다고요.[장소원 국립국어원장]맞습니다. 국립국어원에서는 2020년부터 한글날을 즈음해서 세계 한국어 한마당이라는 행사를 개최해서 말씀하신 대로 학술대회와 한글 한국어 산업전 또 전 국민이 참여하는 문화행사를 합니다.올해 학술대회는 10월 6일과 7일 한국프레스센터에서 한국어학뿐 아니라 한국어 교육과 통번역학과 등 14개 학술단체가 참여해서 '세상을 잇는 한국어, 미래를 이끄는 한글'이라는 주제로 학술대회를 열고요.또 10월 6일부터 국립한글박물관 별관에서는 한글 한국어 산업전을 개최하는데 인공지능과 관련된 6개 기업들이 참여해서 소통의 벽을 허무는 한국어 인공지능이

In [297]:
# show_article(doc_id)